In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.width',1000)

!pip install vecstack

from vecstack import stacking

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=8338b619524e9e3125dd252aaae68053890ce47462ff0a7c6d00ceded38f1e48
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [2]:

trainfile = r'/gdrive/My Drive/house-prices-advanced-regression-techniques/train.csv'

trainData=pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/house-prices-advanced-regression-techniques/test.csv'
testData=pd.read_csv(testfile)

print(trainData.shape)
print(testData.shape)


(1460, 81)
(1459, 80)


In [3]:
#Extract Target Column before doing missing value substitutions and one-hot encoding======
train_target_copy=trainData["SalePrice"]
trainData=trainData.drop(["SalePrice"],axis=1)
print(trainData.shape)

(1460, 80)


In [4]:
#Data Cleaning
#Missing Data
combined_Data = pd.concat([trainData,testData],keys=[0,1])
print(pd.isna(combined_Data).sum())
threshold=int(0.84*(combined_Data.shape[0]))
print(threshold)
td1=combined_Data.dropna(axis=1,thresh=threshold)
print(td1.shape)


Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       486
LotArea             0
Street              0
Alley            2721
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         24
MasVnrArea         23
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           81
BsmtCond           82
BsmtExposure       82
BsmtFinType1       79
BsmtFinSF1          1
BsmtFinType2       80
BsmtFinSF2          1
BsmtUnfSF           1
TotalBsmtSF         1
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [5]:
##Inpute Missing Values
#For numeric columns
train_numeric = td1.select_dtypes(include=['int','float64'])
train_numeric = train_numeric.drop(columns = ['MSSubClass','OverallQual','OverallCond'], axis = 1)
train_actual_categorical = pd.concat([td1['MSSubClass'],td1['OverallQual'],td1['OverallCond']], axis = 1)
print(train_actual_categorical.head())
print(train_numeric.shape)

     MSSubClass  OverallQual  OverallCond
0 0          60            7            5
  1          20            6            8
  2          60            7            5
  3          70            7            5
  4          60            8            5
(2919, 33)


In [6]:
#give mean value for na
numeric = train_numeric.columns
for num in numeric:
  train_numeric[num] = train_numeric[num].fillna(train_numeric[num].median())

#checking missing values in num_combined_data
train_numeric_MV = train_numeric.isnull().sum()
print(train_numeric_MV.T)
print(train_numeric_MV.T.shape)

Id               0
LotArea          0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
dtype: int64
(33,)


In [7]:
#Filling missing values for categorical columns
#Obtaining categorical columns identified by python
train_categorical = td1.select_dtypes(include = ['object'])
#First combining the previously excluded cat variables labeled as numeric by python
cat_combined_data = pd.concat([train_actual_categorical, train_categorical],axis = 1)
print(cat_combined_data.shape)
print("\n")
 
#checking missing values in cat_combined_data
print(cat_combined_data.isnull().sum().to_frame().T)
#checking columns
print(train_categorical.columns)

(2919, 41)


   MSSubClass  OverallQual  OverallCond  MSZoning  Street  LotShape  LandContour  Utilities  LotConfig  LandSlope  Neighborhood  Condition1  Condition2  BldgType  HouseStyle  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  ExterQual  ExterCond  Foundation  BsmtQual  BsmtCond  BsmtExposure  BsmtFinType1  BsmtFinType2  Heating  HeatingQC  CentralAir  Electrical  KitchenQual  Functional  GarageType  GarageFinish  GarageQual  GarageCond  PavedDrive  SaleType  SaleCondition
0           0            0            0         4       0         0            0          2          0          0             0           0           0         0           0          0         0            1            1          24          0          0           0        81        82            82            79            80        0          0           0           1            1           2         157           159         159         159           0         1              0
Index(['MSZoning'

In [8]:
#Filling mode values for nan in categorical columns in the existing NewTestTrainWithColDrop[num]
cat_cols = cat_combined_data.columns
for cols in cat_cols:
    cat_combined_data[cols] = cat_combined_data[cols].fillna(cat_combined_data[cols].mode)
 
#Checking the cat_combined data after filling
print(cat_combined_data.isnull().sum().to_frame().T)
print("\nData\n")
#print(cat_combined_data.head())
print(cat_combined_data.shape)
print("\n\n")

   MSSubClass  OverallQual  OverallCond  MSZoning  Street  LotShape  LandContour  Utilities  LotConfig  LandSlope  Neighborhood  Condition1  Condition2  BldgType  HouseStyle  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  ExterQual  ExterCond  Foundation  BsmtQual  BsmtCond  BsmtExposure  BsmtFinType1  BsmtFinType2  Heating  HeatingQC  CentralAir  Electrical  KitchenQual  Functional  GarageType  GarageFinish  GarageQual  GarageCond  PavedDrive  SaleType  SaleCondition
0           0            0            0         0       0         0            0          0          0          0             0           0           0         0           0          0         0            0            0           0          0          0           0         0         0             0             0             0        0          0           0           0            0           0           0             0           0           0           0         0              0

Data

(2919, 41)





In [0]:
#Hot encoding all the string type/boolean categorical variabes
categoricalFeatures =['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']
train_categorical = pd.get_dummies(train_categorical, columns = categoricalFeatures)

In [10]:
#combining encoded categories with actual categoris 
encoded_cat_columns = pd.concat([train_actual_categorical,train_categorical], axis = 1)
print(encoded_cat_columns.shape)
 
#encoding the numeric categories
num_cat = ['MSSubClass', 'OverallQual', 'OverallCond']
encoded_cat_combined_data = pd.get_dummies(encoded_cat_columns, columns = num_cat)
 
print(encoded_cat_combined_data.head())
print(encoded_cat_combined_data.shape)

(2919, 237)
     MSZoning_C (all)  MSZoning_FV  MSZoning_RH  MSZoning_RL  MSZoning_RM  Street_Grvl  Street_Pave  LotShape_IR1  LotShape_IR2  LotShape_IR3  LotShape_Reg  LandContour_Bnk  LandContour_HLS  LandContour_Low  LandContour_Lvl  Utilities_AllPub  Utilities_NoSeWa  LotConfig_Corner  LotConfig_CulDSac  LotConfig_FR2  LotConfig_FR3  LotConfig_Inside  LandSlope_Gtl  LandSlope_Mod  LandSlope_Sev  Neighborhood_Blmngtn  Neighborhood_Blueste  Neighborhood_BrDale  Neighborhood_BrkSide  Neighborhood_ClearCr  Neighborhood_CollgCr  Neighborhood_Crawfor  Neighborhood_Edwards  Neighborhood_Gilbert  Neighborhood_IDOTRR  Neighborhood_MeadowV  Neighborhood_Mitchel  Neighborhood_NAmes  Neighborhood_NPkVill  Neighborhood_NWAmes  Neighborhood_NoRidge  Neighborhood_NridgHt  Neighborhood_OldTown  Neighborhood_SWISU  Neighborhood_Sawyer  Neighborhood_SawyerW  Neighborhood_Somerst  Neighborhood_StoneBr  Neighborhood_Timber  Neighborhood_Veenker  Condition1_Artery  Condition1_Feedr  Condition1_Norm  \


In [11]:
#combining all the numeric and categorical columns 
final_combined_data = pd.concat([train_numeric,encoded_cat_combined_data], axis = 1)
print(final_combined_data.shape)
 
#checking missing values for the final combined df
pd.DataFrame(final_combined_data.isnull().sum()).T

(2919, 302)


,Id,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CW

In [12]:
#Seperate Train and Test data
x_train = final_combined_data.xs(0)
x_test = final_combined_data.xs(1)
Y_train = train_target_copy
print(x_train.shape)
print(x_train.head())


(1460, 302)
   Id  LotArea  YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  TotRmsAbvGrd  Fireplaces  GarageYrBlt  GarageCars  GarageArea  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold  MSZoning_C (all)  MSZoning_FV  MSZoning_RH  MSZoning_RL  MSZoning_RM  Street_Grvl  Street_Pave  LotShape_IR1  LotShape_IR2  LotShape_IR3  LotShape_Reg  LandContour_Bnk  LandContour_HLS  LandContour_Low  LandContour_Lvl  Utilities_AllPub  Utilities_NoSeWa  LotConfig_Corner  LotConfig_CulDSac  LotConfig_FR2  LotConfig_FR3  LotConfig_Inside  LandSlope_Gtl  LandSlope_Mod  LandSlope_Sev  Neighborhood_Blmngtn  Neighborhood_Blueste  Neighborhood_BrDale  Neighborhood_BrkSide  Neighborhood_ClearCr  Neighborhood_CollgCr  Neighborhood_Crawfor  Neighborhood_Edwards  Neighborhood_Gilbert  Neighborhood_IDOTRR  

In [13]:
#1A Construct Decision Tree Regressor
clf=DecisionTreeRegressor()
clf.fit(x_train,Y_train)
clf_prediction=clf.predict(x_test)

print(clf_prediction.shape)

(1459,)


In [0]:
resDT = pd.DataFrame(clf.predict(x_test), columns = ['SalePrice'])
DT_result = pd.concat([x_test['Id'], resDT], axis = 1)
DT_result.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultDT.csv", index = None)


In [15]:
#1B Construct Decision Tree Regressor with HP
clf_HP=DecisionTreeRegressor()
parameters = {'max_depth':range(10,50,10),'criterion':['mse','friedman_mse'],'min_samples_leaf':range(10,60,5)}
clf_HP_random=RandomizedSearchCV(clf_HP,parameters,n_iter=30,cv=4)
clf_HP_random.fit(x_train,Y_train)
grid_HP_random=clf_HP_random.best_params_
print(grid_HP_random)


{'min_samples_leaf': 15, 'max_depth': 40, 'criterion': 'friedman_mse'}


In [0]:

clf_HP_tuned=DecisionTreeRegressor(**grid_HP_random)
clf_HP_tuned.fit(x_train,Y_train)
clf_HP_tuned_pred=clf_HP_tuned.predict(x_test)

In [0]:
resDT_tuned = pd.DataFrame(clf_HP_tuned.predict(x_test), columns = ['SalePrice'])
DT_result_tuned = pd.concat([x_test['Id'], resDT_tuned], axis = 1)
DT_result_tuned.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resDT_tuned.csv", index = None)

In [18]:
#2A Construct Random forest Regressor
rf=RandomForestRegressor()
rf.fit(x_train,Y_train)
rf_prediction=rf.predict(x_test)

print(rf_prediction.shape)

(1459,)


In [0]:
resRF = pd.DataFrame(rf.predict(x_test), columns = ['SalePrice'])
RF_result = pd.concat([x_test['Id'], resRF], axis = 1)
RF_result.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultRF.csv", index = None)

In [20]:
#2B Construct Random forest Regressor with HP
rf_HP=RandomForestRegressor()
rf_parameters = {'max_depth':range(10,60,10),'n_estimators':[30,100,10],'min_samples_split':range(10,60,10)}
rf_HP_grid=GridSearchCV(rf_HP,rf_parameters,cv=5)
rf_HP_grid.fit(x_train,Y_train)
grid_HP=rf_HP_grid.best_params_
print(grid_HP)


{'max_depth': 40, 'min_samples_split': 10, 'n_estimators': 100}


In [0]:
rf_HP_tuned=RandomForestRegressor(**grid_HP)
rf_HP_tuned.fit(x_train,Y_train)
rf_HP_tuned_pred=rf_HP_tuned.predict(x_test)

In [0]:
resRF_tuned = pd.DataFrame(rf_HP_tuned.predict(x_test), columns = ['SalePrice'])
RF_result_tuned = pd.concat([x_test['Id'], resRF_tuned], axis = 1)
RF_result_tuned.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resRF_tuned.csv", index = None)

In [23]:
#3 Construct MLPRegressor
mlp=MLPRegressor()
mlp.fit(x_train,Y_train)
mlp_prediction=mlp.predict(x_test)

print(mlp_prediction.shape)

(1459,)


In [0]:
resMLP = pd.DataFrame(mlp.predict(x_test), columns = ['SalePrice'])
MLP_result = pd.concat([x_test['Id'], resMLP], axis = 1)
MLP_result.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultMLP.csv", index = None)

In [25]:
#4 Construct SVR
svr=SVR()
svr.fit(x_train,Y_train)
svr_prediction=svr.predict(x_test)

print(svr_prediction.shape)

(1459,)


In [0]:
resSVR = pd.DataFrame(svr.predict(x_test), columns = ['SalePrice'])
SVR_result = pd.concat([x_test['Id'], resSVR], axis = 1)
SVR_result.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultSVR.csv", index = None)

In [27]:
#5A Construct GradientBoostingRegressor
boosting=GradientBoostingRegressor()
boosting.fit(x_train,Y_train)
boosting_prediction=boosting.predict(x_test)

print(boosting_prediction.shape)

(1459,)


In [0]:
resBoosting = pd.DataFrame(boosting.predict(x_test), columns = ['SalePrice'])
Boosting_result = pd.concat([x_test['Id'], resBoosting], axis = 1)
Boosting_result.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultBoosting.csv", index = None)

In [45]:
#5B Construct GradientBoostingRegressor with HP
boosting_HP=GradientBoostingRegressor()
boosting_parameters = {'loss':['ls','lad','huber','quantile'],'n_estimators':[80,120,10],'min_samples_split':[2,10,2]}
boosting_HP_grid=GridSearchCV(boosting_HP,boosting_parameters,cv=7)
boosting_HP_grid.fit(x_train,Y_train)
grid_HP_boosting=boosting_HP_grid.best_params_
print(grid_HP_boosting)

{'loss': 'ls', 'min_samples_split': 10, 'n_estimators': 120}


In [0]:
boosting_HP_tuned=GradientBoostingRegressor(**grid_HP_boosting)
boosting_HP_tuned.fit(x_train,Y_train)
boosting_HP_tuned_pred=boosting_HP_tuned.predict(x_test)

In [0]:
resBoosting_tuned = pd.DataFrame(boosting_HP_tuned.predict(x_test), columns = ['SalePrice'])
Boosting_result_tuned = pd.concat([x_test['Id'], resBoosting_tuned], axis = 1)
Boosting_result_tuned.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultBoosting_tuned.csv", index = None)

In [29]:
#6A Construct SGD
sgd = SGDRegressor(loss = 'squared_loss', penalty = 'l1', max_iter = 800, l1_ratio = 0.14, tol=1e-3)
sgd.fit(x_train, Y_train)
sgd_prediction=sgd.predict(x_test)

print(sgd_prediction.shape)

(1459,)


In [0]:
resSGD = pd.DataFrame(sgd.predict(x_test), columns = ['SalePrice'])
sgd_result = pd.concat([x_test['Id'], resSGD], axis = 1)
sgd_result.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultSGD.csv", index = None)

In [31]:
#6B Construct SGD for HP
sgd_HP=SGDRegressor()
sgd_parameters = {'loss':['squared_loss','huber','epsilon_insensitive','squared_epsilon_insensitive'],'penalty':['l2','l1','elasticnet'],'max_iter':range(800,1200,100),'l1_ratio':[0.13,0.17,0.01]}
sgd_HP_grid=GridSearchCV(sgd_HP,sgd_parameters,cv=5)
sgd_HP_grid.fit(x_train,Y_train)
grid_HP_sgd=sgd_HP_grid.best_params_
print(grid_HP_sgd)


{'l1_ratio': 0.01, 'loss': 'huber', 'max_iter': 1100, 'penalty': 'l2'}


In [0]:
sgd_HP_tuned=SGDRegressor(**grid_HP_sgd)
sgd_HP_tuned.fit(x_train,Y_train)
sgd_HP_tuned_pred=sgd_HP_tuned.predict(x_test)

In [0]:
resSGD_tuned = pd.DataFrame(sgd_HP_tuned.predict(x_test), columns = ['SalePrice'])
SGD_result_tuned = pd.concat([x_test['Id'], resSGD_tuned], axis = 1)
SGD_result_tuned.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resSGD_tuned.csv", index = None)

In [49]:
#Stacking
models = [ clf_HP, rf_HP, boosting_HP]
      
S_Train, S_Test = stacking(models,                   
                           x_train, Y_train, x_test,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
                                        
                           n_folds=4, 
                                                    
                           verbose=2)

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [DecisionTreeRegressor]
    fold  0:  [30095.32054795]
    fold  1:  [31650.81643836]
    fold  2:  [28157.29315068]
    fold  3:  [29457.84109589]
    ----
    MEAN:     [29840.31780822] + [1257.15871316]
    FULL:     [29840.31780822]

model  1:     [RandomForestRegressor]
    fold  0:  [19519.36767123]
    fold  1:  [21306.83369863]
    fold  2:  [18891.65506849]
    fold  3:  [20513.66712329]
    ----
    MEAN:     [20057.88089041] + [924.35157843]
    FULL:     [20057.88089041]

model  2:     [GradientBoostingRegressor]
    fold  0:  [16287.74044964]
    fold  1:  [18307.28014432]
    fold  2:  [15735.48611213]
    fold  3:  [16085.93139998]
    ----
    MEAN:     [16604.10952652] + [1002.98239678]
    FULL:     [16604.10952652]



In [50]:
#1A Stacked model(Gradient Boosting)
boosting_stacked=GradientBoostingRegressor()
boosting_stacked.fit(S_Train,Y_train)
boosting_stacked_pred=boosting_stacked.predict(S_Test)
print(boosting_stacked_pred.shape)

(1459,)


In [0]:
resBoosting_stack = pd.DataFrame(boosting_stacked.predict(S_Test), columns = ['SalePrice'])
Boosting_result_stack = pd.concat([x_test['Id'], resBoosting_stack], axis = 1)
Boosting_result_stack.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultBoosting_stack.csv", index = None)

In [57]:
#1B Stacked model(Gradient Boosting with HP)
boosting_HP_stack=GradientBoostingRegressor()
boosting_parameters_stack = {'loss':['ls','lad','huber','quantile'],'n_estimators':[90,110,5],'min_samples_split':[2,10,2]}
boosting_HP_grid_stack=GridSearchCV(boosting_HP_stack,boosting_parameters_stack,cv=5)
boosting_HP_grid_stack.fit(S_Train,Y_train)
grid_HP_boosting_stack=boosting_HP_grid_stack.best_params_
print(grid_HP_boosting_stack)

{'loss': 'lad', 'min_samples_split': 10, 'n_estimators': 90}


In [0]:
boosting_HP_tuned_stack=GradientBoostingRegressor(**grid_HP_boosting_stack)
boosting_HP_tuned_stack.fit(S_Train,Y_train)
boosting_HP_tuned_pred_stack=boosting_HP_tuned_stack.predict(S_Test)

In [0]:
resBoosting_tuned_stack = pd.DataFrame(boosting_HP_tuned_stack.predict(S_Test), columns = ['SalePrice'])
Boosting_result_tuned_stack = pd.concat([x_test['Id'], resBoosting_tuned_stack], axis = 1)
Boosting_result_tuned_stack.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/Boosting_result_tuned_stack.csv", index = None)

In [52]:
#2 Random Forest Regressor over stack
rf_stack = RandomForestRegressor()
rf_stack.fit(S_Train, Y_train)
RF_stacked_pred=rf_stack.predict(S_Test)
print(RF_stacked_pred.shape)

(1459,)


In [0]:
resRF_stack = pd.DataFrame(rf_stack.predict(S_Test), columns = ['SalePrice'])
RF_result_stack = pd.concat([x_test['Id'], resRF_stack], axis = 1)
RF_result_stack.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultRF_stack.csv", index = None)

In [54]:
#3 Decision Tree Regressor over stack
DT_stack = DecisionTreeRegressor()
DT_stack.fit(S_Train, Y_train)
DT_stacked_pred=DT_stack.predict(S_Test)
print(DT_stacked_pred.shape)

(1459,)


In [0]:
resDT_stack = pd.DataFrame(DT_stack.predict(S_Test), columns = ['SalePrice'])
DT_result_stack = pd.concat([x_test['Id'], resDT_stack], axis = 1)
DT_result_stack.to_csv("/gdrive/My Drive/house-prices-advanced-regression-techniques/resultDT_stack.csv", index = None)